In [36]:
# !pip install fpdf

import torch
from PIL import Image
import os
import io
import tempfile
import random
from transformers import BlipProcessor, BlipForConditionalGeneration
from gtts import gTTS
import base64
import gdown
from fpdf import FPDF
import pygame


In [25]:
# # Initialize OpenAI API 
# def get_env():
#     api_key = os.getenv('OPENAI_API_KEY')
#     if not api_key:
#         raise ValueError("API key not found. Please set the OPENAI_API_KEY environment variable.")
#     return api_key



In [ ]:
# Initialize the processor and model
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
# model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base"
# !pip install gdown
# Function to load the trained model, download it if necessary
# Function to load the trained model, download it if necessary
def load_trained_model():
    model_path = "model/blip_image_captioning_model.pth"
    drive_link = "https://drive.google.com/drive/u/1/folders/1484tKLarTxDIIie_MkYWqh9Y7tbhiR-f"

    # Download the model if it does not exist
    if not os.path.exists(model_path):
        os.makedirs("model", exist_ok=True)
        gdown.download(drive_link, model_path, quiet=False)

    # Load the base model architecture from Hugging Face
    base_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
    # Load the custom state dict into the base model
    state_dict = torch.load(model_path, map_location=torch.device('cpu'))
    base_model.load_state_dict(state_dict)
    return base_model

model = load_trained_model()

def get_caption(image_path):
    image = Image.open(image_path)
    inputs = processor(image, return_tensors="pt")
    out = model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)
    return caption

In [27]:
# def text_to_speech(text, output_file):
#     # OpenAI TTS request (assuming you have a TTS method)
#     client = OpenAI()
#     response = client.audio.speech.create(
#     model="tts-1",
#     voice="alloy",
#     input=get_caption,
# )

#     audio_data = response['audio']
#     with open(output_file, 'wb') as f:
#         f.write(audio_data)

# def save_model(model, path):
#     torch.save(model.state_dict(), path)

In [37]:
def text_to_speech(text):
    tts = gTTS(text=text, lang='en')
    audio_fp = io.BytesIO()
    tts.write_to_fp(audio_fp)
    audio_fp.seek(0)
    return audio_fp

def save_model(model, path):
    torch.save(model.state_dict(), path)

In [38]:
# Function to load images from a folder
def load_images(folder_path):
    # List to store image file paths
    image_files = []

    # Supported image extensions
    supported_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif')

    # Iterate over files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.lower().endswith(supported_extensions):
            image_files.append(os.path.join(folder_path, file_name))

    return image_files

In [40]:
def main():
    # Specify the folder path containing images
    folder_path = r'C:\Users\User\OneDrive\Desktop\DATA SCIENCE\GEN_ai\VisoVox_Grok\images'
    image_files = load_images(folder_path)
    if not image_files:
        print("No images found in the specified folder.")
        return

    # Select a random image from the folder
    random_image_path = random.choice(image_files)
    print(f"Selected image: {random_image_path}")

    # Get caption for the image
    caption = get_caption(random_image_path)
    if caption is None:
        print("Could not generate a caption for the image.")
        return
    print("Caption:", caption)

    # Convert caption to speech and get the audio file path
    audio_fp = text_to_speech(caption)

    # Initialize pygame mixer
    pygame.mixer.init()

    # Load and play the audio file
    pygame.mixer.music.load(audio_fp, 'mp3')
    pygame.mixer.music.play()

    # Wait for the audio to finish playing
    while pygame.mixer.music.get_busy():
        pygame.time.Clock().tick(10)

# Call the main function
main()

Selected image: C:\Users\User\OneDrive\Desktop\DATA SCIENCE\GEN_ai\VisoVox_Grok\images\antlers.jpg


Caption: a group of deers
